In [4]:
import pandas as pd
import numpy as np
from numpy import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.feature_selection import mutual_info_classif
import pickle
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np

In [3]:
filenames = ['MachineLearningCSV/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', 'MachineLearningCSV/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', 'MachineLearningCSV/Friday-WorkingHours-Morning.pcap_ISCX.csv', 'MachineLearningCSV/Monday-WorkingHours.pcap_ISCX.csv','MachineLearningCSV/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', 'MachineLearningCSV/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', 'MachineLearningCSV/Tuesday-WorkingHours.pcap_ISCX.csv', 'MachineLearningCSV/Wednesday-workingHours.pcap_ISCX.csv']
df_aug = pd.read_csv(filenames[0])
for name in filenames[1:]:
  df = pd.read_csv(name)
  df_aug = pd.concat([df_aug, df], axis=0)
  
print(df_aug.shape)
df_aug.to_csv('IDS-2017-full.csv', index=False)

(2830743, 79)


In [2]:
def corr_df(x, corr_val):
    '''
    Obj: Drops features that are strongly correlated to other features.
          This lowers model complexity, and aids in generalizing the model.
    Inputs:
          df: features df (x)
          corr_val: Columns are dropped relative to the corr_val input (e.g. 0.8)
    Output: df that only includes uncorrelated features
    '''

    # Creates Correlation Matrix and Instantiates
    corr_matrix = x.corr()
    print(corr_matrix.shape)
    print(len(corr_matrix.columns))
    iters = range(len(corr_matrix.columns) -1 )
    drop_cols = []

    # Iterates through Correlation Matrix Table to find correlated columns
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = item.values
            if abs(val) >= corr_val:
                # Prints the correlated feature set and the corr val
                print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])
                
    

    drop_cols = set(drop_cols)
    print(drop_cols)
    cols = [col for col in x.columns if col not in drop_cols]
    
    #no_drops = [i for i in range(len(corr_matrix.columns)) if i not in drops]
    # Drops the correlated columns
    #for i in drops:
    #    col = x.iloc[:, (i+1):(i+2)].columns.values
    #    x = x.drop(col, axis=1)
   # return x
    return x[cols]

In [5]:
#Read
print('Reading data ...')
df = pd.read_csv('IDS-2017-full.csv',)
print('Reading done')

RANDOM_SEED = 41

le = LabelEncoder()
df[' Label'] = le.fit_transform(df[' Label'])

Y = df[' Label']
X = df.drop([' Label'], axis=1)

del df
X=X.astype('float64')
print(X.select_dtypes('float64').shape)

#Replace inf values (with NaN ?)
X=X.replace([np.inf, -np.inf], np.nan)

#Fill NaN
X.fillna(0.0, inplace=True)

#Train test split

print('Splitting ...')
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state = RANDOM_SEED, stratify=Y)
print('Spliiting done.')
del X
del Y
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

# Eliminate mutually correlated feature pairs

print('Correlated feature elimination')

X_train = corr_df(X_train, 1.00)
print(X_train.columns)
X_test = X_test[X_train.columns]

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

# Scale

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))

#print('MI classif ...')
#mi = np.array(mutual_info_classif(X_train, Y_train, n_neighbors=5, copy=True))
#X_train = X_train[X_train.columns[mi.argsort()[-39:][::-1]]]
#X_test = X_test[X_test.columns[mi.argsort()[-39:][::-1]]]

#print ('Chi Square ...')
#chi, _ = np.array(chi2(X_train, Y_train))
#X_train = X_train[X_train.columns[chi.argsort()[-35:][::-1]]]
#X_test = X_test[X_test.columns[chi.argsort()[-35:][::-1]]]

#print('F classif')
#chi, _ = np.array(f_classif(X_train, Y_train))
#X_train = X_train[X_train.columns[chi.argsort()[-39:][::-1]]]
#X_test = X_test[X_test.columns[chi.argsort()[-39:][::-1]]]

#feature_indices = np.array([33, 34, 46,  4, 47,  8, 32, 48,  5,  0, 14,  1, 40, 28, 10, 17, 15,
#       29, 12, 30, 11, 16,  9, 13,  7, 22,  3, 19, 50, 20,  2, 54, 55, 18,
#       21, 51, 53, 23, 31,  6, 49, 37, 38, 52, 35, 24, 44, 45, 42, 41, 36,
#       25, 39, 43, 26, 27])
#print('Select features')
#X_train = X_train[feature_indices[0:25]]
#X_test = X_test[feature_indices[0:25]]




Reading data ...
Reading done
(2830743, 78)
Splitting ...
Spliiting done.
(1896597, 78)
(934146, 78)
(1896597,)
(934146,)
Correlated feature elimination
(78, 78)
78
 SYN Flag Count | Fwd PSH Flags | 1.0
 CWE Flag Count |  Fwd URG Flags | 1.0
 Avg Fwd Segment Size |  Fwd Packet Length Mean | 1.0
 Avg Bwd Segment Size |  Bwd Packet Length Mean | 1.0
 Fwd Header Length.1 |  Fwd Header Length | 1.0
Subflow Fwd Packets |  Total Fwd Packets | 1.0
 Subflow Bwd Packets |  Total Backward Packets | 1.0
{' SYN Flag Count', ' Fwd Header Length.1', ' Subflow Bwd Packets', 'Subflow Fwd Packets', ' Avg Bwd Segment Size', ' CWE Flag Count', ' Avg Fwd Segment Size'}
Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length